In [19]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis import network as net
from pyvis.network import Network
import random

from serpapi import GoogleScholarSearch
import json
import requests

## ES 1
A partire da nodes.csv, utilizzare la libreria Python SerpAPI per scaricare, per ciascuno dei 7 autori elencati:
- author_id: ID identificativo del profilo Google Scholar
- cited_by: numero totale di citazioni ricevute
- interests: elenco degli interessi di ciascun autore
Sfruttando la libreria Python Pandas, usare la struttura dati DataFrame per aggiornare il file originale con apposite colonne e memorizzarlo nella cartella /data.

In [17]:
apikey ={
    'SERP_API_KEY_FAVA': "1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048", 
    'SERP_API_KEY_LIGUGNANA': "11a077a8bee5bb1c2a25af914046bc7ce53050b3416e5720239e43223c63374b",
    'SERP_API_KEY_PARAVANO' :  "42b4919fd319f0ff32c19ae31a88b4ff67a544e29ca2a017905088b2ae11e60e",
    'SERP_API_KEY_TAMMARO': "6eaf5a9a0d2eef8bf026a947a3dffbb0d31cb1d6e122b81c0e6062f6b23a4c40"
}
apikey = "1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048"
def key_choice():
    return random.choice(list(apikey.items()))[1]
#for che cambi la key se da errore la chiamata


In [13]:
key_choice()


'11a077a8bee5bb1c2a25af914046bc7ce53050b3416e5720239e43223c63374b'

In [109]:
def list_profile(apiKey, name, affiliation):
    base_url = "https://serpapi.com/search"
    
    params = {
        "api_key": apiKey,
        "engine": "google_scholar_profiles",
        "hl": "en",
        "mauthors": name
    }      
    
    response = requests.get(base_url, params = params)
    results = response.json()
    
    authorId = results["profiles"][0]["author_id"] #save author_id from of the author
    citedBy = results["profiles"][0]["cited_by"]
    interests = results["profiles"][0]["interests"]
    
    '''    
    print(affiliation)
    print(name)'''
    
    
    for profile in results["profiles"]:
        print(profile["affiliations"])
        print(profile["name"])
        if profile["affiliations"] == affiliation and profile["name"] == name:
            tuplas = tuple((name,affiliation,authorId,citedBy,interests))
            df = pd.DataFrame({'name': [tuplas[0]],
                       'affiliations': [tuplas[1]],
                       'author_id': [tuplas[2]],
                       'cited_by': [tuplas[3]],
                       'interests': [tuplas[4]]})
            return df
    
    #verifies the affiliation, the profile found is the one that i'm searching for
    
   

In [110]:

print(list_profile(apikey, 'David La Barbera', 'Università Degli Studi Di Udine'))

Università Degli Studi Di Udine
David La Barbera
               name                     affiliations     author_id  cited_by  \
0  David La Barbera  Università Degli Studi Di Udine  MvN8wKEAAAAJ       139   

                                           interests  
0  [{'title': 'Computer Science', 'serpapi_link':...  


In [97]:
print(chr(224))

à


In [75]:


df_nodes = pd.read_csv(r"Data/nodes.csv")

df_temp = pd.DataFrame(columns=['name','affiliations','author_id','cited_by','interests'])

for index, row in df_nodes.iterrows():
    df_temp=pd.concat([df_temp,(list_profile(apikey, row[0], row[1]))])
    ##df_temp.concat(list_profile(apikey, row[0], row[1]))
df_temp

NameError: name 'profiles' is not defined

,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,,NaN,NaN
1,Michael Soprano,Postdoctoral Research Fellow at the University...,,NaN,NaN
2,Kevin Roitero,University of Udine,,NaN,NaN
3,Stefano Mizzaro,Full professor of Computer Science and Informa...,,NaN,NaN
4,Damiano Spina,"School of Computing Technologies, RMIT University",,NaN,NaN
5,Gianluca Demartini,Associate Professor at the University of Queen...,,NaN,NaN
6,Eddy Maddalena,"Università degli Studi di Udine, Italy",,NaN,NaN


## 2. Analisi dei Coautori tramite SerpAPI su Google Scholar

### a. Ottenere i Coautori dei 7 Autori Principali

Per ciascuno dei 7 autori, utilizzare il suo ID per accedere al relativo profilo Google Scholar e scaricare l'elenco dei suoi coautori utilizzando SerpAPI. Con tale elenco di nomi:;

#### a.1. Ricerca dei Coautori su Google Scholar

Utilizzare le SerpAPI per cercare su Google Scholar un ricercatore che corrisponde a ciascun nome nella lista di coautori. Per ogni risultato, salvare le informazioni come `name`, `affiliations`, `author_id`, `cited_by`, e `interests` in un nuovo DataFrame contenente tutte queste informazioni relative ai coautori dei 7 autori originari.

### b. Concatenazione dei DataFrame

Concatenare il DataFrame contenente le informazioni sui 7 autori principali con quello dei coautori generato al punto 2a in un unico DataFrame.

### c. Creazione del DataFrame delle Relazioni di Co-Authorship

Creare un terzo DataFrame con le colonne `author1` e `author2` che rappresenta le co-authorship. In questo DataFrame, una riga rappresenta un arco di coauthorship tra due autori. La co-authorship è binaria, non pesata.

#### Esempio

Se David La Barbera e Michael Soprano sono coautori, una riga del DataFrame creato al punto 2c rappresenterà questa relazione.

### Risultati Finali

A questo punto, saranno stati prodotti due DataFrame:

- Uno contenente le informazioni relative agli autori (originali + i relativi coautori), con le colonne `name`, `affiliations`, `cited_by`, `interests`. Salvare come `nodes.csv` nella cartella `/data` tale DataFrame.

- Un altro per le relazioni di co-authorship dai 7 autori principali verso i relativi coautori, con colonne `author1` e `author2`. Salvare come `edges.csv` nella cartella `/data` tale DataFrame.





In [ ]:
def get_name_coauthors(apiKey, authorID):
    
    base_url = "https://serpapi.com/search"
    
    params = {
        "api_key": apiKey,
        "engine": "google_scholar_author",
        "hl": "en",
        "author_id": authorID
    }
    
    response = requests.get(base_url, params = params)
    results = response.json()
    
    return #dataset

In [16]:


df_Coauthors = pd.DataFrame(columns=['name','affiliations','author_id','cited_by','interests'])

file = open('all_profiles_for_larry_page.json')
res = json.load(file)


,name,affiliations,author_id,cited_by,interests
0,David La Barbera,Università Degli Studi Di Udine,MvN8wKEAAAAJ,139,"[{'title': 'Computer Science', 'serpapi_link':..."


In [ ]:


apikey = {
    'SERP_API_KEY_FAVA': "1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048",
    'SERP_API_KEY_LIGUGNANA': "11a077a8bee5bb1c2a25af914046bc7ce53050b3416e5720239e43223c63374b",
    'SERP_API_KEY_PARAVANO': "42b4919fd319f0ff32c19ae31a88b4ff67a544e29ca2a017905088b2ae11e60e",
    'SERP_API_KEY_TAMMARO': "6eaf5a9a0d2eef8bf026a947a3dffbb0d31cb1d6e122b81c0e6062f6b23a4c40"
}
apikey = "1ae8a37d9b3dabb5732fcd7a9cb57410194766951242a4f4ac5bbd9fb7012048"


def key_choice():
    return random.choice(list(apikey.items()))[1]


#for che cambi la key se da errore la chiamata

key_choice()


def list_profile(apiKey, name, affiliation):
    base_url = "https://serpapi.com/search"

    params = {
        "api_key": apiKey,
        "engine": "google_scholar_profiles",
        "hl": "en",
        "mauthors": name
    }

    search = GoogleScholarSearch(params)
    profiles = search.get_dict()
    profili = profiles["profiles"]


    for profilo in profili:
        print(f'Cheave: {profilo}, Valore: {profilo}')
    
    



    response = requests.get(base_url, params=params)
    results = response.json()

    authorId = results["profiles"][0]["author_id"]  #save author_id from of the author
    citedBy = results["profiles"][0]["cited_by"]
    interests = results["profiles"][0]["interests"]

    print(affiliation)
    print(name)

    for profile in results["profiles"]:
        print(profile["affi"])
        if profile["affiliations"] == affiliation and profile["name"] == name:
            print('mario')
            tuple = tuple((name, affiliation, authorId, citedBy, interests))
            df = pd.DataFrame({'name': [tuple[0]],
                               'affiliations': [tuple[1]],
                               'author_id': [tuple[2]],
                               'cited_by': [tuple[3]],
                               'interests': [tuple[4]]})
            return df

